In [9]:
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from sklearn.model_selection import train_test_split

## 2.3.5 결정 트리
- 분류와 회귀 문제에 널리 사용되는 모델
- 결정에 다다르기 위해 예/아니오 질문을 통해 학습 (스무고개 놀이 질문과 비슷)